
<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Image source: https://www.kaggle.com/
</p>

## Stage 1: Installing dependencies and setting up GPU environment

In [ ]:
!pip install tensorflow
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 20 kB/s 
     |████████████████████████████████| 439 kB 59.8 MB/s 
     |████████████████████████████████| 6.0 MB 58.0 MB/s 
     |████████████████████████████████| 1.7 MB 61.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Fo

## Stage 2: Import dependencies for the project

In [ ]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
tf.__version__

'2.11.0'

## Stage 3: Dataset preprocessing



### Loading the dataset

In [ ]:
#Loading the Fashion Mnist dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
assert X_train.shape == (60000, 28, 28)
assert X_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

4422102/4422102 [==============================] - 0s 0us/step


### Image normalization

We devide each image in the training and testing dataset with the maxiumum number of pixels (255).

In this way each pixel will be in the rainge [0, 1]. By normalizing imaes we are making sure that our model (ANN) trains faster.

In [ ]:
X_train = X_train / 255.0

In [ ]:
X_test = X_test / 255.0

### Reshaping of the dataset

Since we are using fully connected network, we reshape the training and testing subsets to be in the vector format. 

In [ ]:
#Since each image is 28x28, we simply use reshape the full dataset to [-1 (all elements), height * width]
X_train = X_train.reshape(-1, 28*28)

In [ ]:
X_train.shape

(60000, 784)

In [ ]:
#Reshape the testing subset in the same way
X_test = X_test.reshape(-1, 28*28)

## Stage 4: Building an Artificial Neural network

### Making MISH

See def

In [ ]:
def mish(inputs):
    x = tf.nn.softplus(inputs)
    x = tf.nn.tanh(x)
    x = tf.multiply(x, inputs)
    return x

### Defining the model

Simply define an object of the Sequential model.

In [ ]:
model = tf.keras.models.Sequential()

### Adding the first layer (Dense layer)

Layer hyper-parameters:
- number of units/neurons: 128
- activation function: ReLU
- input_shape: (784, )

In [ ]:
model.add(tf.keras.layers.Dense(units=784, activation=mish, input_shape=(784, )))

### Second Layer

In [ ]:
model.add(tf.keras.layers.Dense(units=392, activation=mish ))

### Adding a Dropout layer 

Dropout is a Regularization technique where we randomly set neurons in a layer to zero. In this way, while training those neurons won't be updated. Because some percentage of neurons won't be updated the whole training process is long and we have less chance for overfitting.

In [ ]:
model.add(tf.keras.layers.Dropout(0.34))

### Adding the second layer (output layer)

- units == number of classes (10 in the case of Fashion MNIST)
- activation = 'softmax'

In [ ]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Comiling the model

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy 

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 392)               307720    
                                                                 
 dropout (Dropout)           (None, 392)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                3930      
                                                                 
Total params: 927,090
Trainable params: 927,090
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 10s 3ms/step - loss: 0.4857 - sparse_categorical_accuracy: 0.8232
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3720 - sparse_categorical_accuracy: 0.8638
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3330 - sparse_categorical_accuracy: 0.8762
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3062 - sparse_categorical_accuracy: 0.8853
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2825 - sparse_categorical_accuracy: 0.8938
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2656 - sparse_categorical_accuracy: 0.9018
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2524 - sparse_categorical_accuracy: 0.9043
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2383 - sparse_categorical_accuracy: 0.9106
Epoch 9/10
1875/1875 [=========

### Model evaluation and prediction

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3384 - sparse_categorical_accuracy: 0.8839


In [ ]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.883899986743927


## Stage 5 : Saving the model

### Saving the architecture (topology) of the network

In [ ]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Saving network weights

In [ ]:
model.save_weights("fashion_model.h5")